# Assignment 4: Apriori Algorithm

## Task 2

In [1]:
# importing relevant libraries
import pandas as pd
import numpy as np
import os 
import csv
from itertools import combinations
from itertools import permutations

### Frequent itemsets generation

In [2]:
# the support function below generates an adjacency 
# matrix between transactions (rows) and items (columns)
# if item is present in the transaction it is market as 1 else 0
def get_vocab(data_dict):
    # define dictionary
    vocabulary = {}
    # generate frequency count of all items in data 
    for i in data_dict:
        for j in data_dict[i]:
            if j in vocabulary:
                vocabulary[j].append(i)
            else:
                vocabulary[j] = [i]
    # create aforementioned matrix
    vertical_matrix = np.matrix([[0 for i in range(0,len(vocabulary))] for i in range(0, len(data_dict))])            
    # map keys to indecies of the matrix
    mapping = {}
    keys = list(vocabulary.keys())
    for i in range(len(keys)):
        mapping[i] = keys[i]
    for i in mapping:
        instancies = vocabulary[mapping[i]]
        for j in instancies:
            vertical_matrix[j,i] = 1
    # assign a matrix row for each item in the vocabulary
    for i in mapping:
        vocabulary[mapping[i]] = vertical_matrix[:,i]
    return vocabulary

In [3]:
# this support function gets candidate combinations
# for size k+1 from input combinations of size k
def get_word_pairs(frequent_itemsets):
    available_keys = set()
    # find possible keys that can be added to the combinations of size k
    for i in frequent_itemsets:
        a = i.split(',')
        for j in a:
            available_keys.add(j)
    key_set = (set(available_keys)) 
    new_itemset = []
    # create all combinations of k+1 size 
    for i in frequent_itemsets:
        new_i = ""
        for j in key_set:
            if j not in str(i):
                a = i.split(',')
                a.append(j)
                a = sorted(a)
                new_i = a[0]
                for k in range(1,len(a)):
                    new_i += ',' + a[k]
                new_itemset.append(new_i)
    # return candidate combinations 
    new_itemset = list(set(new_itemset))
    return new_itemset
# this support function generates frequent itemsets 
def get_min_subset(data, min_support, T):
    # preprocess and get vocab from data 
    vocab = get_vocab(data)
    word_set = list(vocab.keys())
    min_subset = {}
    frequent_itemsets = []
    l = len(vocab)
    # check is sets of size 1 are frequent 
    for i in word_set:
        if int(sum(vocab[i])) >= min_support:
            frequent_itemsets.append(i)
            min_subset[i] = int(sum(vocab[i]))
    # create frequent itemsets from sets of size k>=2
    for i in range(0,l):
        new_set = get_word_pairs(frequent_itemsets)
        frequent_itemsets = []
        counter = 0
        length = 0 
        # for every candidate value
        for value in new_set:
            v = value.split(',')
            base_letter = v[0]       
            # check the frequency of the item 
            vector = vocab[base_letter]
            for other in v[1:]:
                # check the frequency of the item
                row = vocab[other]
                # compute where the items occure sumultaneouly (intersection)
                vector = ((row+vector)==2)*1
            # check if min support condition is satisfied 
            if int(sum(vector))/T >= min_support:
                frequent_itemsets.append(value)
                vocab[value]=vector
                # add to frequent itemsets 
                min_subset[value] = int(sum(vector))
                length += 1
            # keep counter for large data processing
            counter += 1
            if counter % 1000 == 0:
                print(counter, l)
    # return frequent itemsets 
    return min_subset

### Rule Generation

In [4]:
# The main function of the algorithm that generates rules and frequent itemsets 
def apriori(data, minsup, minconf, minlift):
    # getting vocab and frequent itemsets
    vocab = get_vocab(data)
    word_set = list(vocab.keys())
    min_subset = get_min_subset(data, minsup, len(data)) # min460
    confident_itemsets = {}
    l = len(vocab)
    l_ws = len(word_set)
    k = l_ws -1
    rules = {}
    T = len(data)
    # iterating through frequent itemsets to generate rules
    for frequent_itemset in list(min_subset.keys()):
        isl = frequent_itemset.split(',')
        union = min_subset[frequent_itemset]
        # for any sets of size 2 use different method
        if len(isl) ==2:
            for i in range(len(isl)):
                conf = union/min_subset[isl[i]]
                lift = conf / min_subset[isl[(i+1)%len(isl)]]
                if conf >= minconf:
                    if lift >= minlift:
                        rule =  isl[i] + '>' + isl[(i+1)%len(isl)]
                        rules[rule] = [conf, lift]
        # for any larger sets get combinations of different length
        # and check which combinations satisfy the requirements
        for n in range(1,len(isl)):
            cmb = list(combinations(isl, n))
            alt_cmb = list(combinations(isl, len(isl)-n))
            for m in cmb:
                m_list = ','.join(m)
                for a in alt_cmb:
                    equal=False
                    a_list = ','.join(a)
                    for item in m:
                        if item in a:
                            equal = True
                    # checking if satisfy lift and confidence conditions
                    if equal == False:
                        conf = union / min_subset[m_list]
                        lift = conf / (min_subset[a_list]/T) 
                        if conf >= minconf:
                            if lift >= minlift:
                                rule = m_list + '>' + a_list
                                # creating a dictionary 
                                rules[rule] = {'conf':conf, "lift":lift, "nitems":len(isl), 'support':union/T}

    # sorting the dictionary first by number of items, then by lift, then by confidence and ultimately by support
    sorted_tuples = sorted(rules.items(), reverse=True, key=lambda item: (item[1]['nitems'],item[1]['lift'],item[1]['conf'],item[1]['support']))
    sorted_dict = {k: v for k, v in sorted_tuples}
    # returing the rules
    return sorted_dict

## Task 3

In [5]:
# Defining data from task 1
task1_data = {0:['A','B','C','D','F'], 
              1:['A','B','C','D'], 
              2:['A','B','C','D'], 
              3:['A','B'], 
              4:['B','C','E']}


In [6]:
# getting results for task 3
task3_result = apriori(data=task1_data, minsup=0.15, minconf=0.8, minlift=0)

Below are the task 1 rules with corresponding confidence, support and lift for each rule 

In [7]:
# printing the rules and their parameters 
task3_result

{'F>A,B,C,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'A,F>B,C,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'B,F>A,C,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'C,F>A,B,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'D,F>A,B,C': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'A,B,F>C,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'A,C,F>B,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'B,C,F>A,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'B,D,F>A,C': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'A,B,C,F>D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 5,
  'support': 0.2},
 'A,D,F>B,C': {'conf': 1.0, 'lift': 1.25, 'nitems': 5, 'support': 0.2},
 'C,D,F>

Below are the just the rules for the dataset for task 1

In [8]:
len(list(task3_result.keys())) # total number of rules 

97

In [9]:
list(task3_result.keys()) # printing the rules themselves

['F>A,B,C,D',
 'A,F>B,C,D',
 'B,F>A,C,D',
 'C,F>A,B,D',
 'D,F>A,B,C',
 'A,B,F>C,D',
 'A,C,F>B,D',
 'B,C,F>A,D',
 'B,D,F>A,C',
 'A,B,C,F>D',
 'A,D,F>B,C',
 'C,D,F>A,B',
 'A,B,D,F>C',
 'B,C,D,F>A',
 'A,C,D,F>B',
 'D>A,B,C',
 'A,C>B,D',
 'B,D>A,C',
 'A,B,C>D',
 'F>A,B,D',
 'A,F>B,D',
 'B,F>A,D',
 'A,B,F>D',
 'F>B,C,D',
 'B,F>C,D',
 'C,F>B,D',
 'B,C,F>D',
 'F>A,B,C',
 'B,F>A,C',
 'F>A,C,D',
 'A,F>C,D',
 'C,F>A,D',
 'D,F>A,C',
 'A,C,F>D',
 'A,D>B,C',
 'C,D>A,B',
 'A,B,D>C',
 'B,C,D>A',
 'D,F>A,B',
 'B,D,F>A',
 'D,F>B,C',
 'B,D,F>C',
 'A,F>B,C',
 'C,F>A,B',
 'A,B,F>C',
 'B,C,F>A',
 'A,D,F>C',
 'C,D,F>A',
 'A,C,D>B',
 'A,D,F>B',
 'C,D,F>B',
 'A,C,F>B',
 'D>A,C',
 'A,C>D',
 'F>B,D',
 'B,F>D',
 'F>A,C',
 'F>A,D',
 'A,F>D',
 'F>C,D',
 'C,F>D',
 'D>B,C',
 'B,D>C',
 'A,D>C',
 'C,D>A',
 'D>A,B',
 'B,D>A',
 'F>B,C',
 'B,F>C',
 'A,F>C',
 'C,F>A',
 'F>A,B',
 'B,F>A',
 'D,F>A',
 'E>B,C',
 'B,E>C',
 'D,F>C',
 'C,D>B',
 'A,C>B',
 'A,D>B',
 'C,F>B',
 'D,F>B',
 'A,F>B',
 'C,E>B',
 'F>D',
 'D>A',
 'D>C',
 '

## Task 4

In [10]:
# reading in the data 
testing_file = open('supermarket.csv', 'r')
lines = testing_file.readlines()
nrows = len(lines)

In [11]:
# constructing dictionary to store the data
data_dict = {}
totals = {}
for i in range(len(lines)):
    params = lines[i].strip().split(',')
    total = params.pop(-1).split(' = ')[1]
    data_dict[i] = sorted(params)
    totals[i] = total

Running apriori algorithm on supermarket data using minsup=0.1,0.15 and 0.2, minconf=0.8, minlift=0. After the algorithm has completed running, the data is stored in a csv file with minsup=0.1 rules stored in "results_01_08.csv", minsup=0.15 rules stored in "results_015_08.csv" and minsup=0.2 rules stored in "results_02_08.csv".

In [12]:
result_dict = apriori(data_dict, 0.15, 0.8, 0) # appying the apriori algorithm

1000 122
2000 122
3000 122
4000 122
5000 122
6000 122
7000 122
1000 122
2000 122
3000 122
4000 122
5000 122
1000 122
2000 122
3000 122
4000 122
5000 122
6000 122
7000 122
8000 122
9000 122
10000 122
1000 122
2000 122
3000 122
4000 122
5000 122


Writting the csv file to store the results

In [13]:
import os 
import csv
a_file = open("results_015_08.csv", "w")

writer = csv.writer(a_file)
for key, value in result_dict.items():
    
    row = [key] + [value[i] for i in value]
    
    writer.writerow(row)

a_file.close()

Importing back the results into a dataframe from csv

In [14]:
columns = ['Rule', 'Confidence', 'Lift', 'Number of Elements in rule', 'Support']

In [15]:
rules_01_08 = pd.read_csv('results_02_08.csv', header=None)
rules_01_08.columns = columns
rules_01_08

,Rule,Confidence,Lift,Number of Elements in rule,Support
0,"biscuits,bread and cake,frozen foods,vegetable...",0.829464,1.295723,5,0.200778
1,"biscuits,bread and cake,frozen foods,fruit>veg...",0.812773,1.270079,5,0.200778
2,"baking needs,bread and cake,fruit,milk-cream>v...",0.808786,1.263847,5,0.202939
3,"baking needs,bread and cake,milk-cream,vegetab...",0.803251,1.254774,5,0.202939
4,"biscuits,frozen foods,fruit,vegetables>bread a...",0.894129,1.242383,5,0.200778
...,...,...,...,...,...
181,"frozen foods,soft drinks>bread and cake",0.802372,1.114887,3,0.219365
182,small goods>bread and cake,0.835125,1.160398,2,0.201426
183,canned fruit>bread and cake,0.810600,1.126320,2,0.224768
184,jams-spreads>bread and cake,0.803599,1.116593,2,0.221958


In [16]:
rules_015_08 = pd.read_csv('results_015_08.csv', header=None)
rules_015_08.columns = columns
rules_015_08

,Rule,Confidence,Lift,Number of Elements in rule,Support
0,"baking needs,biscuits,bread and cake,frozen fo...",0.844340,1.318960,6,0.154744
1,"baking needs,biscuits,bread and cake,frozen fo...",0.838407,1.310136,6,0.154744
2,"baking needs,biscuits,frozen foods,fruit,veget...",0.899497,1.249842,6,0.154744
3,"bread and cake,frozen foods,juice-sat-cord-ms,...",0.806674,1.432815,5,0.151502
4,"baking needs,bread and cake,frozen foods,party...",0.805585,1.430880,5,0.162092
...,...,...,...,...,...
732,small goods>bread and cake,0.835125,1.160398,2,0.201426
733,canned fruit>bread and cake,0.810600,1.126320,2,0.224768
734,jams-spreads>bread and cake,0.803599,1.116593,2,0.221958
735,canned fish-meat>bread and cake,0.801275,1.113364,2,0.162957


In [17]:
rules_02_08 = pd.read_csv('results_02_08.csv', header=None)
rules_02_08.columns = columns
rules_02_08

,Rule,Confidence,Lift,Number of Elements in rule,Support
0,"biscuits,bread and cake,frozen foods,vegetable...",0.829464,1.295723,5,0.200778
1,"biscuits,bread and cake,frozen foods,fruit>veg...",0.812773,1.270079,5,0.200778
2,"baking needs,bread and cake,fruit,milk-cream>v...",0.808786,1.263847,5,0.202939
3,"baking needs,bread and cake,milk-cream,vegetab...",0.803251,1.254774,5,0.202939
4,"biscuits,frozen foods,fruit,vegetables>bread a...",0.894129,1.242383,5,0.200778
...,...,...,...,...,...
181,"frozen foods,soft drinks>bread and cake",0.802372,1.114887,3,0.219365
182,small goods>bread and cake,0.835125,1.160398,2,0.201426
183,canned fruit>bread and cake,0.810600,1.126320,2,0.224768
184,jams-spreads>bread and cake,0.803599,1.116593,2,0.221958


## Task 5

In [18]:
# support function identical to task 2
def get_word_pairs_task5(frequent_itemsets):
    available_keys = set()
    for i in frequent_itemsets:
        a = i.split(',')
        for j in a:
            available_keys.add(j)
    #print((available_keys))
    key_set = (set(available_keys)) 
    new_itemset = []
    for i in frequent_itemsets:
        new_i = ""
        for j in key_set:
            if j not in str(i):
                a = i.split(',')
                a.append(j)
                #print(a)
                a = sorted(a)
                new_i = a[0]
                for k in range(1,len(a)):
                    new_i += ',' + a[k]
                #new_i = ",".join(sorted(list(new_i)))
                new_itemset.append(new_i)
    new_itemset = list(set(new_itemset))
    return new_itemset
# support function with minrelativesup implemented 
def get_min_subset_task5(data, min_support,minrelativesup, T):
    vocab = get_vocab(data)
    word_set = list(vocab.keys())
    min_subset = {}
    frequent_itemsets = []
    min_relsub = {}
    l = len(vocab)
    maxsubset = 0
    # calculating the first maxsubset value and storing it 
    for i in word_set:
        if int(sum(vocab[i])) >= min_support:
            frequent_itemsets.append(i)
            min_subset[i] = int(sum(vocab[i]))
            if int(sum(vocab[i])) > maxsubset:
                    maxsubset = int(sum(vocab[i]))
    # iterating over subsets of size k>=2
    for i in range(0,l):
        new_set = get_word_pairs_task5(frequent_itemsets)
        frequent_itemsets = []
        print(len(new_set), 'set len')
        counter = 0
        length = 0 
        temp_itemset = {}
        new_max_subset =0
        for value in new_set:
            v = value.split(',')
            base_letter = v[0]                  
            vector = vocab[base_letter]
            for other in v[1:]:
                row = vocab[other]
                vector = ((row+vector)==2)*1
            # checking if the minimum support is satisfied 
            if int(sum(vector))/T >= min_support:
                # calculating the maxsubset for set size k
                if int(sum(vector)) >= new_max_subset:
                    new_max_subset = int(sum(vector))
                # storing parameters for end of loop comparison
                temp_itemset[value] = [vector,int(sum(vector))]
                length += 1
            counter += 1
            if counter % 1000 == 0:
                print(counter, l)
        # performing the maxrelativesup comparison
        for value in temp_itemset:
            # comparing with respect to maxsubset of sets of size k-1
            if temp_itemset[value][1] / maxsubset >= minrelativesup:
                # updating the frequent itemsets and resetting teh maxsubset value
                # with new kth maxsubset value
                frequent_itemsets.append(value)
                vocab[value]=temp_itemset[value][0]
                min_subset[value] = temp_itemset[value][1]
                min_relsub[value] = temp_itemset[value][1]/maxsubset
                maxsubset = new_max_subset
        print(i, l)
    # returing frequent itemset and minrelativesup statistics
    return min_subset, min_relsub

# main algorithm for rule geneation and frequent itemset generation similar to task 2
def apriori_task5(data, minsup, minconf, minlift, minrelativesup):
    vocab = get_vocab(data)
    word_set = list(vocab.keys())
    min_subset, min_relsub = get_min_subset_task5(data, minsup, minrelativesup, len(data)) # min460
    print(min_relsub)
    confident_itemsets = {}
    l = len(vocab)
    l_ws = len(word_set)
    k = l_ws -1
    rules = {}
    T = len(data)
    for frequent_itemset in list(min_subset.keys()):
        isl = frequent_itemset.split(',')
        union = min_subset[frequent_itemset]
        if len(isl) ==2:
            for i in range(len(isl)):
                conf = union/min_subset[isl[i]]
                lift = conf / min_subset[isl[(i+1)%len(isl)]]
                if conf >= minconf:
                    if lift >= minlift:
                        rule =  isl[i] + '>' + isl[(i+1)%len(isl)]
                        rules[rule] = [conf, lift]

        for n in range(1,len(isl)):
            cmb = list(combinations(isl, n))
            alt_cmb = list(combinations(isl, len(isl)-n))
            for m in cmb:
                m_list = ','.join(m)
                
                for a in alt_cmb:
                    equal=False
                    a_list = ','.join(a)
                    for item in m:
                        if item in a:
                            equal = True
                    if equal == False and (a_list in min_subset) and (m_list in min_subset):
                        conf = union / min_subset[m_list]
                        lift = conf / (min_subset[a_list]/T) 
                        if conf >= minconf:
                            if lift >= minlift:
                                rule = m_list + '>' + a_list
                                rules[rule] = {'conf':conf, "lift":lift, "nitems":len(isl), 'support':union/T}

    # returning sorted dictionary
    sorted_tuples = sorted(rules.items(), reverse=True, key=lambda item: (item[1]['nitems'],item[1]['lift'],item[1]['conf'],item[1]['support']))
    sorted_dict = {k: v for k, v in sorted_tuples}
    return sorted_dict

In [19]:
task5_result = apriori_task5(data=task1_data, minsup=0, minconf=0.8, minlift=0, minrelativesup=0.3)

15 set len
0 6
4 set len
1 6
1 set len
2 6
0 set len
3 6
0 set len
4 6
0 set len
5 6
{'A,B': 0.8, 'B,D': 0.75, 'A,D': 0.75, 'B,C': 1.0, 'A,C': 0.75, 'C,D': 0.75, 'A,B,C': 0.75, 'B,C,D': 1.0, 'A,B,D': 1.0, 'A,C,D': 1.0, 'A,B,C,D': 1.0}


In [20]:
task5_result

{'D>A,B,C': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 4,
  'support': 0.6},
 'A,C>B,D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 4,
  'support': 0.6},
 'B,D>A,C': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 4,
  'support': 0.6},
 'A,B,C>D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 4,
  'support': 0.6},
 'A,D>B,C': {'conf': 1.0, 'lift': 1.25, 'nitems': 4, 'support': 0.6},
 'C,D>A,B': {'conf': 1.0, 'lift': 1.25, 'nitems': 4, 'support': 0.6},
 'A,B,D>C': {'conf': 1.0, 'lift': 1.25, 'nitems': 4, 'support': 0.6},
 'B,C,D>A': {'conf': 1.0, 'lift': 1.25, 'nitems': 4, 'support': 0.6},
 'A,C,D>B': {'conf': 1.0, 'lift': 1.0, 'nitems': 4, 'support': 0.6},
 'D>A,C': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 3,
  'support': 0.6},
 'A,C>D': {'conf': 1.0,
  'lift': 1.6666666666666667,
  'nitems': 3,
  'support': 0.6},
 'D>B,C': {'conf': 1.0, 'lift': 1.25, 'nitems': 3, 'support': 0.6},
 'B,D>C': {'conf': 1.0, 'lift': 1.25, 'nitems':